## 1-加载数据集

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
%matplotlib inline
import seaborn as sns
 
base_path = './datasets/titanic/'

train = pd.read_csv(base_path+'train.csv')
test = pd.read_csv(base_path+'test.csv')

## 2-初步探索

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

* **PassengerId**: a unique identifier for each passenger
* **Survived**: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* **Pclass**: passenger class.
* **Name**, **Sex**, **Age**: self-explanatory
* **SibSp**: how many siblings & spouses of the passenger aboard the Titanic.
* **Parch**: how many children & parents of the passenger aboard the Titanic.
* **Ticket**: ticket id
* **Fare**: price paid (in pounds)
* **Cabin**: passenger's cabin number
* **Embarked**: where the passenger embarked the Titanic

从结果可以看到，训练数据集有891个样本（样本量不大，要在模型训练过程中小心过拟合），11个特征和1个标签，其中特征‘Age'，'Cabin'，'Embarked'都有不同程度的缺损；测试集有418个样本，只有11个特征，其中特征'Age'，'Fare'，'Cabin'有不同程度的缺损。

对这些缺损的数据可以选择的处理方式由简到难包括：
1. 直接删除此特征（缺损数据太多的情况，防止引入噪声）
2. 直接删除缺损数据的样本（~土豪操作~只用于训练数据集，且样本量较大，缺损数据样本较少的情况）
3. 直接将有无数值作为新的特征（数据缺失较多，且数据有无本身是对预测是一个有用的特征）
4. 中值或均值回补（缺失数据较多，不想损失此较多训练数据，特征又比较重要的情况，是比较常用的方法）
5. 参考其他特征，利用与此特征的相关性编写算法回补数据（~大神级操作~回补的准确性可能会比较高一些，但实现过程复杂）

这几种方法具体使用哪一个需要根据实际情况决定，选用复杂的方法得到的结果不一定就好。

再来观察这11个特征的类型：
* 其中有4个特征：'PassengerId'，'Pclass’,'Sibsp'，'Parch'属于**整数型数据**；
* 5个特征：'Name'，'Sex'，'Ticket'，'Cabin'，'Embarked'属于**字符串类型数据**；
* 2个特征：'Age'，'Fare'属于**浮点数**。

然而这些数据格式并不都是机器学习模型的菜，不能直接喂给模型字符串数据。
为统一数据格式，方便模型训练，我们下面还需要对这些特征数据进行缩放和转化。

## 3-特征分析与处理
有些可能要用python的**map函数进行特征分层**，非常实用！

* 乘客阶级

In [ ]:
#采用seaborn绘图函数库作可视化分析
sns.countplot(x="Pclass", hue="Survived", data=train)

* 性别

In [ ]:
sns.countplot(x="Sex", hue="Survived", data=train)

* 年龄（缺失值用no表示）

In [ ]:
train['Age'].isnull().sum() #统计缺失值个数

In [ ]:
#将有年龄数值的转化为yes,缺损的转化为no
train['Age']=train['Age'].map(lambda x:'yes' if 0<x<100 else 'no')
sns.countplot(x="Age", hue="Survived", data=train)

In [ ]:
train=pd.read_csv('./datasets/titanic/train.csv')
sns.violinplot(x='Survived',y='Age',data=train)  #小提琴图

In [ ]:
train['Age']=train['Age'].map(lambda x: 'child' if x<12 
            else 'youth' if x<30 
            else 'adlut' if x<60 
            else 'old' if x<75 
            else 'tooold' if x>=75 
            else 'null')

* 兄弟姐妹数

In [ ]:
sns.countplot(x="SibSp", hue="Survived", data=train)

In [ ]:
train['SibSp']=train['SibSp'].map(lambda x: 'small' if x<1 
                else 'middle' if x<3 
                else 'large')

* 父母孩子数

In [ ]:
sns.countplot(x="Parch", hue="Survived", data=train)

In [ ]:
train['Parch']=train['Parch'].map(lambda x: 'small' if x<1 
                else 'middle' if x<4 
                else 'large')

* 船票价格

In [ ]:
sns.violinplot(x='Survived',y='Fare',data=train)

In [ ]:
#因为原图效果不明显，做对数变换
train['Fare']=train['Fare'].map(lambda x:np.log(x+1))  #用numpy库里的对数函数对Fare的数值进行对数转换
sns.violinplot(x='Survived',y='Fare',data=train)

* 船票价格

In [ ]:
train['Fare']=train['Fare'].map(lambda x: 'poor' if x<2.5 else 'rich')

* 船舱编号

In [ ]:
train['Cabin']=train['Cabin'].map(lambda x:'yes' if type(x)==str else 'no')
sns.countplot(x="Cabin", hue="Survived", data=train)

* 上船港口

In [ ]:
train['Embarked'].value_counts()

In [ ]:
sns.countplot(x="Embarked", hue="Survived", data=train)

In [ ]:
train['Embarked'].isnull().sum()

In [ ]:
train.dropna(axis=0,inplace=True) #删掉含有缺损值的样本

数据大致浏览完成，查看现在训练集的信息

In [ ]:
train.info()

至此特征分析全部完成，可以删除的特征：'PassengerId'，'Name'和'Ticket'。(更专业的人可以选择对这几个特征再做处理)

然后对剩余的特征进行独热编码。

In [ ]:
#将训练数据分成标记和特征两部分
labels= train['Survived']
features= train.drop(['Survived','PassengerId','Name','Ticket'],axis=1)

#对所有特征实现独热编码
features = pd.get_dummies(features)
encoded = list(features.columns)
print ("{} total features after one-hot encoding.".format(len(encoded)))

* 对测试集进行同样的处理。

In [ ]:
test.head()

In [ ]:
#对'Age','SibSp'，'Parch'特征分段分类
test['Age']=test['Age'].map(lambda x: 'child' if x<12 else 'youth' if x<30 else 'adlut' if x<60 else 'old' if x<75 else 'tooold' if x>=75 else 'null')
test['SibSp']=test['SibSp'].map(lambda x: 'small' if x<1 else 'middle' if x<3 else 'large')
test['Parch']=test['Parch'].map(lambda x: 'small' if x<1 else 'middle' if x<4 else 'large')
#均值补齐'Fare'特征值对数转换和分类
test.Fare.fillna(test['Fare'].mean(), inplace=True)
test['Fare']=test['Fare'].map(lambda x:np.log(x+1))
test['Fare']=test['Fare'].map(lambda x: 'poor' if x<2.5 else 'rich')
#按'Cabin'是否缺损分类
test['Cabin']=test['Cabin'].map(lambda x:'yes' if type(x)==str else 'no')
#删除不需要的特征并进行独热编码
Id=test['PassengerId']
test=test.drop(['PassengerId','Name','Ticket'],axis=1)
test=pd.get_dummies(test)
encoded = list(test.columns)
print ("{} total features after one-hot encoding.".format(len(encoded)))

## 4-模型构建

考虑要用到的算法包括：决策树，SVM，随机森林，Adaboost，KNN以及传说中的大杀器Xgboost。

**先建立一个统一的训练框架**，方便之后**网格搜索**调参。

**这个思路很好，可以借鉴学习！！**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score,roc_auc_score
from time import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier

# 通用函数框架
def fit_model(alg,parameters):
    X=features
    y=labels  #由于数据较少，使用全部数据进行网格搜索
    scorer=make_scorer(roc_auc_score)  #使用roc_auc_score作为评分标准
    grid = GridSearchCV(alg,parameters,scoring=scorer,cv=5)  #使用网格搜索，出入参数
    start=time()  #计时
    grid=grid.fit(X,y)  #模型训练
    end=time()
    t=round(end-start,3)
    print (grid.best_params_)  #输出最佳参数
    print ('searching time for {} is {} s'.format(alg.__class__.__name__,t)) #输出搜索时间
    return grid #返回训练好的模型

#列出需要使用的算法
alg1=DecisionTreeClassifier(random_state=29)
alg2=SVC(probability=True,random_state=29)  #由于使用roc_auc_score作为评分标准，需将SVC中的probability参数设置为True
alg3=RandomForestClassifier(random_state=29)
alg4=AdaBoostClassifier(random_state=29)
alg5=KNeighborsClassifier(n_jobs=-1)
alg6=XGBClassifier(random_state=29,n_jobs=-1,use_label_encoder=False)

# 列出需要调整的参数范围
#第一版-与第二版不同处备注：
# parameters1={'max_depth':range(1,10),'min_samples_split':range(2,10)}
# parameters2 = {"C":range(1,20), "gamma": [0.05,0.1,0.15,0.2,0.25]}
# parameters3_2 = {'max_depth':range(1,10),'min_samples_split':range(2,10)}  #搜索空间太大，分两次调整参数
# parameters5 = {'n_neighbors':range(2,10),'leaf_size':range(10,80,20) }

#第二版-
parameters1={'max_depth':range(1,10),'min_samples_split':range(1,10)}
parameters2 = {"C":range(1,20), "gamma": [0.01,0.02,0.05,0.1,0.15]}
parameters3_1 = {'n_estimators':range(10,200,10)}
parameters3_2 = {'max_depth':range(1,10),'min_samples_split':range(1,8)}  #搜索空间太大，分两次调整参数
parameters4 = {'n_estimators':range(10,200,10),'learning_rate':[i/10.0 for i in range(5,15)]}
parameters5 = {'n_neighbors':range(2,10),'leaf_size':range(5,60,20)  }
parameters6_1 = {'n_estimators':range(10,200,10)}
parameters6_2 = {'max_depth':range(1,10),'min_child_weight':range(1,10)}
parameters6_3 = {'subsample':[i/10.0 for i in range(1,10)], 
                'colsample_bytree':[i/10.0 for i in range(1,10)]}#搜索空间太大，分三次调整参数

**调参范围（可能并非最优，还可以改进）**

列出我们需要调整的参数及取值范围，这是一个很繁琐的工作，需要大量的尝试和优化。

接下来**开始调参**（炼丹）

1 决策树

In [ ]:
clf1=fit_model(alg1,parameters1) 

2 SVM

In [ ]:
clf2=fit_model(alg2,parameters2) 

3 随机森林

In [ ]:
clf3_m1=fit_model(alg3,parameters3_1) #第一次调参

In [ ]:
alg3=RandomForestClassifier(random_state=29,n_estimators=180) #第二次调参
clf3=fit_model(alg3,parameters3_2)

4 AdaBoost

In [ ]:
clf4=fit_model(alg4,parameters4)

5 KNN

In [ ]:
clf5=fit_model(alg5,parameters5)

6 Xgboost

In [ ]:
clf6_m1=fit_model(alg6,parameters6_1) #第一次调参

In [ ]:
alg6=XGBClassifier(n_estimators=140,random_state=29,n_jobs=-1) #第二次调参
clf6_m2=fit_model(alg6,parameters6_2)

In [ ]:
alg6=XGBClassifier(n_estimators=140,max_depth=4,min_child_weight=5,random_state=29,n_jobs=-1) #第三次调参
clf6=fit_model(alg6,parameters6_3)

## 4-验证结果

定义一个保存函数，将预测的结果保存为可以提交的格式；然后调用这个函数，完成6个模型的预测。

In [ ]:
base_pth = './datasets/titanic/'

def save(clf,i):
    pred=clf.predict(test)
    sub=pd.DataFrame({ 'PassengerId': Id, 'Survived': pred })
    sub.to_csv(base_pth + "res_tan_{}.csv".format(i), index=False)
    
i=1
for clf in [clf1,clf2,clf3,clf4,clf5,clf6]:
    save(clf,i)
    i=i+1

最后将个模型预测结果csv文件提交至kaggle。

-------------------------

* 本文参考作者链接：https://blog.csdn.net/aicanghai_smile/article/details/79234172
* 特此感谢！
  
参考博主的预测成绩**（和我有点不同）**：6个预测结果都超过了0.77，基本达到预测的效果，成绩最好的是随机森林模型，得分0.79425，比较出乎意外的是Xgboost的算法成绩竟然只有0.77511，可能是参数没有调好。大家可以尝试其他参数，说不定可以得到更好的成绩。不过考虑到这个项目数据量太小，能到0.8左右的成绩应该已经比较好了，重要的是学习数据处理，特征分析以及模型构建调参的过程，目的已经达到。

12.23更新：我的第二版预测结果见 ./dataset/titanic/result.png
------------------------

